# Бинарная классификация временных рядов при помощи LSTM

## Импорт необходимых библиотек

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    roc_curve,
    roc_auc_score,
    precision_recall_curve,
    average_precision_score,
    f1_score
)
from sklearn.utils.class_weight import compute_class_weight

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

import warnings
warnings.filterwarnings('ignore')


In [57]:
test = pd.read_parquet("test.parquet")
submission = pd.read_csv('submission.csv')

## Обработка признаков

In [58]:
# Заполнение NaN в 'values' нулями
test['values'] = test['values'].apply(lambda x: [0.0 if pd.isnull(v) else v for v in x])

# Создание маски: 1 если значение присутствует, 0 если заполнено (0.0)
test['mask'] = test['values'].apply(lambda x: [0 if v == 0.0 else 1 for v in x])

In [59]:
# Функция для преобразования дат в временные метки
def convert_dates_to_timestamps(dates_array):
    if isinstance(dates_array, np.ndarray):
        dates_list = dates_array.tolist()
    else:
        dates_list = dates_array

    timestamps = []
    for date_item in dates_list:
        if isinstance(date_item, datetime.date) and not isinstance(date_item, datetime.datetime):
            date_obj = datetime.datetime.combine(date_item, datetime.time())
        elif isinstance(date_item, datetime.datetime):
            date_obj = date_item
        elif isinstance(date_item, str):
            try:
                date_obj = datetime.datetime.strptime(date_item, '%Y-%m-%d')
            except ValueError:
                date_obj = pd.to_datetime(date_item, errors='coerce').to_pydatetime()
                if pd.isnull(date_obj):
                    raise ValueError(f"Не удалось преобразовать строку в дату: {date_item}")
        else:
            raise ValueError(f"Неизвестный формат даты: {date_item} (тип {type(date_item)})")
        timestamp = date_obj.timestamp()
        timestamps.append(timestamp)
    return timestamps


In [60]:
# Применение функции к столбцу 'dates'
test['timestamps'] = test['dates'].apply(convert_dates_to_timestamps)

In [61]:
# Объединение значений, временных меток и маски
def combine_features(row):
    values = row['values']
    timestamps = row['timestamps']
    mask = row['mask']

    combined = [[v, t, m] for v, t, m in zip(values, timestamps, mask)]
    return combined

In [62]:
test['combined'] = test.apply(combine_features, axis=1)

In [63]:
# Создание списков входных данных
X = test['combined'].tolist()

In [64]:
# Находим максимальную длину последовательностей
max_len = max(len(seq) for seq in X)

# Собираем все значения и временные метки для масштабирования
all_values = [value for seq in X for value, _, _ in seq]
all_timestamps = [timestamp for seq in X for _, timestamp, _ in seq]

# Масштабирование
scaler_values = StandardScaler()
scaler_values.fit(np.array(all_values).reshape(-1, 1))

scaler_timestamps = StandardScaler()
scaler_timestamps.fit(np.array(all_timestamps).reshape(-1, 1))




StandardScaler()

In [65]:
# Функция для масштабирования последовательностей
def scale_sequence(sequence, scaler_values, scaler_timestamps):
    values = [item[0] for item in sequence]
    timestamps = [item[1] for item in sequence]
    mask = [item[2] for item in sequence]

    # Масштабирование значений и временных меток
    values_scaled = scaler_values.transform(np.array(values).reshape(-1, 1)).flatten()
    timestamps_scaled = scaler_timestamps.transform(np.array(timestamps).reshape(-1, 1)).flatten()

    return [[v, t, m] for v, t, m in zip(values_scaled, timestamps_scaled, mask)]

In [66]:
# Применение масштабирования к X
X_scaled = [scale_sequence(seq, scaler_values, scaler_timestamps) for seq in X]

In [67]:

def pad_sequences_pytorch(sequences, max_len, pad_value=0.0):
    padded_sequences = []
    lengths = []
    for seq in sequences:
        seq_len = len(seq)
        lengths.append(seq_len)
        if seq_len < max_len:
            # Дополнение [0.0, 0.0, 0.0] в конце
            padded_seq = seq + [[pad_value, pad_value, 0.0]] * (max_len - seq_len)
        else:
            padded_seq = seq[:max_len]
        padded_sequences.append(padded_seq)
    return torch.tensor(padded_sequences, dtype=torch.float32), torch.tensor(lengths, dtype=torch.long)



In [68]:
# Паддинг последовательностей
X_padded, lengths = pad_sequences_pytorch(X_scaled, max_len, pad_value=0.0)

In [69]:
print("Форма X_padded:", X_padded.shape)  # (num_samples, max_len, 3)
print("Тип данных X_padded:", X_padded.dtype)  # float32

print("Форма lengths:", lengths.shape)  # (num_samples,)
print("Тип данных lengths:", lengths.dtype)  # long

Форма X_padded: torch.Size([20000, 97, 3])
Тип данных X_padded: torch.float32
Форма lengths: torch.Size([20000])
Тип данных lengths: torch.int64


In [70]:
class SequenceDataset(Dataset):
    def __init__(self, sequences, lengths, labels=None):
        self.sequences = sequences
        self.lengths = lengths
        self.labels = labels

    def __len__(self):
        return len(self.sequences)

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.sequences[idx], self.lengths[idx], self.labels[idx]
        else:
            return self.sequences[idx], self.lengths[idx]  # Возвращаем только данные и длины, если метки отсутствуют

In [71]:
test_dataset = SequenceDataset(X_padded, lengths)

## Модель

In [72]:
class LSTMClassifier(torch.nn.Module):
    def __init__(self, input_dim=3, hidden_dim=64, num_layers=1, dropout=0.5):
        super(LSTMClassifier, self).__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers

        self.lstm = torch.nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=False
        )
        self.dropout = torch.nn.Dropout(dropout)
        self.fc = torch.nn.Linear(hidden_dim, 1)

    def forward(self, x, lengths):
        packed_input = torch.nn.utils.rnn.pack_padded_sequence(x, lengths.cpu(), batch_first=True, enforce_sorted=False)
        packed_output, (hn, cn) = self.lstm(packed_input)
        hn = hn[-1]  # Получаем выход последнего слоя LSTM
        out = self.dropout(hn)
        out = self.fc(out)
        return out.squeeze()


In [73]:
# Загрузка сохранённой модели
model = LSTMClassifier(input_dim=3, hidden_dim=64, num_layers=1, dropout=0.5)
model.load_state_dict(torch.load('best_model.pth'))
model.eval()

LSTMClassifier(
  (lstm): LSTM(3, 64, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=64, out_features=1, bias=True)
)

In [74]:
# Использование `test_dataset` для предсказаний
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [75]:
# Выполнение прогнозов с вероятностями
all_probabilities = []
with torch.no_grad():
    for X_batch, lengths_batch in test_loader:
        outputs = model(X_batch, lengths_batch)
        probabilities = torch.sigmoid(outputs)  # Вероятности для классов
        all_probabilities.extend(probabilities.numpy())



In [76]:
# Создание DataFrame с исходными индексами и вероятностями
submission = pd.DataFrame({
    'prediction': all_probabilities
}, index=test['id'])  # Используем оригинальные индексы из test_data

# Сохранение вероятностей в файл
submission.to_csv("submission.csv", index=True)  # Сохраняем с исходными индексами
print("Предсказания сохранены в submission.csv")

Предсказания сохранены в submission.csv
